In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(31):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                type_ = html.xpath('//span[@id="lblPartTypeName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                application_summary = ''.join(html.xpath('//div[@id="divAppSummary"]/p/text()')).strip().replace(', ', '\n')
                
                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@id="imgProductMain"]/@src')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                vehicle_url = html.xpath('//input[@id="hidfDetailApp"]/@value')
                if len(vehicle_url) == 0:
                    vehicle_url = ''
                else:
                    vehicle_url = 'https://www.dormanproducts.com/' + vehicle_url[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                oe = html.xpath('//section[@id="productOE"]/descendant::tbody/tr/th/text()')
                oe = ';'.join([_.strip() for _ in oe])
                
                # = = = = = = = = = = = = = = =
                
                list_tr = html.xpath('//section[@id="productSpec"]/descendant::tr')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Part_Number': part_number,
                                         'Type': type_,
                                         'Description': description,
                                         'Application_Summary': application_summary,
                                         'Vehicle': '',
                                         'OE': oe,
                                         'Url': url,
                                         'Vehicle_Url': vehicle_url,
                                         'Src': src}])
                
                for tr in list_tr:
                    df_temp.loc[0, tr.xpath('./th/text()')[0].strip()[:-1].lower()] = tr.xpath('./td/text()')[0].strip()

                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：392

https://www.dormanproducts.com/p-50429-577-812.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：387
https://www.dormanproducts.com/p-134363-574-228.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：386
https://www.dormanproducts.com/p-50410-577-825.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：385
https://www.dormanproducts.com/p-50054-577-806.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：384
https://www.dormanproducts.com/p-132567-574-024.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：383
https://www.dormanproducts.com/p-7797-577-930.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：382
https://www.dormanproducts.com/p-34670-577-907.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：381
https://www.dormanproducts.com/p-50047-577-807.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：380
https:

https://www.dormanproducts.com/p-75328-577-301.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：319
https://www.dormanproducts.com/p-84611-577-328.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：318
https://www.dormanproducts.com/p-84444-577-329.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：317
https://www.dormanproducts.com/p-51699-577-092.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：316
https://www.dormanproducts.com/p-84180-577-352.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：315
https://www.dormanproducts.com/p-50029-577-093.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：314
https://www.dormanproducts.com/p-96632-577-353.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：313
https://www.dormanproducts.com/p-73523-577-252.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：312
https://www.dorm

https://www.dormanproducts.com/p-75340-577-278.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：252
https://www.dormanproducts.com/p-67534-577-109.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：251
https://www.dormanproducts.com/p-73933-577-214.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：250
https://www.dormanproducts.com/p-50421-577-078.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：249
https://www.dormanproducts.com/p-73948-577-221.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：248
https://www.dormanproducts.com/p-85544-577-379.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：247
https://www.dormanproducts.com/p-50817-577-085.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：246
https://www.dormanproducts.com/p-50499-577-079.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：245
https://www.dorm

https://www.dormanproducts.com/p-141534-574-073.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：185
https://www.dormanproducts.com/p-133437-574-051.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：184
https://www.dormanproducts.com/p-134355-574-120.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：183
https://www.dormanproducts.com/p-141313-574-103.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：182
https://www.dormanproducts.com/p-134350-574-091.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：181
https://www.dormanproducts.com/p-132360-574-020.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：180
https://www.dormanproducts.com/p-132253-574-014.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：179
https://www.dormanproducts.com/p-132361-574-021.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：178
https://

https://www.dormanproducts.com/p-73565-577-238.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：118
https://www.dormanproducts.com/p-84114-577-333.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：117
https://www.dormanproducts.com/p-73536-577-226.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：116
https://www.dormanproducts.com/p-75305-577-269.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：115
https://www.dormanproducts.com/p-68658-577-174.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：114
https://www.dormanproducts.com/p-75092-577-239.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：113
https://www.dormanproducts.com/p-73539-577-240.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：112
https://www.dormanproducts.com/p-75225-577-274.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：111
https://www.dorm

https://www.dormanproducts.com/p-73541-577-212.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：51
https://www.dormanproducts.com/p-75093-577-242.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：50
https://www.dormanproducts.com/p-75281-577-255.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：49
https://www.dormanproducts.com/p-133946-577-142.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：48
https://www.dormanproducts.com/p-50422-577-082.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：47
https://www.dormanproducts.com/p-50815-577-077.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：46
https://www.dormanproducts.com/p-50822-577-076.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：45
https://www.dormanproducts.com/p-90705-577-019.aspx?parttype=Fuel%2520Filler%2520Neck&origin=keyword  <->  [ok] - 剩余数量：44
https://www.dormanprodu